In [21]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import sys
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, Activation, BatchNormalization
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras import callbacks
from keras.callbacks import ModelCheckpoint
import numpy as np

In [0]:

"""
Parameters
"""
epochs = 100
img_width, img_height =227,227
classes_num = 5
batch_size=32
arsitektur = 'AlexNet'
#lr = 0.0004

In [24]:
DATA_DIR = '/content/gdrive/My Drive/CITS Color'

train_datagen = ImageDataGenerator(rescale=1./255,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training') # set as training data

validation_generator = train_datagen.flow_from_directory(
    DATA_DIR, # same directory as training data
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation') # set as validation data

label_map = (train_generator.class_indices)
print(label_map)



Found 1079 images belonging to 5 classes.
Found 268 images belonging to 5 classes.
{'Alang Alang': 0, 'Dracaena': 1, 'Rumput Bambu': 2, 'Rumput Gajah': 3, 'Ruscus': 4}


In [25]:


# (3) Create a sequential model
model = Sequential()

# 1st Convolutional Layer
model.add(Convolution2D(filters=96, input_shape=(img_width,img_height,3), kernel_size=(11,11),\
 strides=(4,4), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation before passing it to the next layer
model.add(BatchNormalization())

# 2nd Convolutional Layer
model.add(Convolution2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Convolutional Layer
model.add(Convolution2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 4th Convolutional Layer
model.add(Convolution2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Batch Normalisation
model.add(BatchNormalization())

# 5th Convolutional Layer
model.add(Convolution2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
model.add(Activation('relu'))
# Pooling
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
# Batch Normalisation
model.add(BatchNormalization())

# Passing it to a dense layer
model.add(Flatten())
# 1st Dense Layer
model.add(Dense(4096, input_shape=(img_width*img_height*3,)))
model.add(Activation('relu'))
# Add Dropout to prevent overfitting
model.add(Dropout(0.5))
# Batch Normalisation
model.add(BatchNormalization())

# 2nd Dense Layer
model.add(Dense(4096))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.5))
# Batch Normalisation
model.add(BatchNormalization())

# 3rd Dense Layer
model.add(Dense(1000))
model.add(Activation('relu'))
# Add Dropout
model.add(Dropout(0.4))
# Batch Normalisation
model.add(BatchNormalization())

# Output Layer
model.add(Dense(classes_num))
model.add(Activation('softmax'))

model.summary()


architecure_dir = os.path.join('/content/gdrive/My Drive/Colab Notebooks/LeafClassification/',arsitektur)
if not os.path.exists(architecure_dir):
  os.mkdir(architecure_dir)
# Save the model architecture
with open(os.path.join(architecure_dir,'architecture.json'), 'w') as f:
    f.write(model.to_json())



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_14 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
batch_normalization_13 (Batc (None, 27, 27, 96)        384       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 23, 23, 256)       614656    
_________________________________________________________________
activation_15 (Activation)   (None, 23, 23, 256)       0         
___________________________

In [0]:
from keras.models import model_from_json
with open(os.path.join(architecure_dir,'architecture.json'), 'r') as f:
    model = model_from_json(f.read())

In [0]:
# (4) Compile
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [28]:
"""
Tensorboard log
"""

log_dir = os.path.join(architecure_dir, 'tf-log')
tb_cb = callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0, write_graph=True, write_images=True)
cbks = [tb_cb]

"""model.fit_generator(
    train_generator,
    samples_per_epoch=samples_per_epoch,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=cbks,
    validation_steps=validation_steps)"""

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    callbacks=cbks,
    epochs = epochs,
    shuffle = True)

#model.fit_generator(datagen.flow(x_train, y_train, batch_size=32), steps_per_epoch=len(x_train) / 32, epochs=epochs)

target_dir =os.path.join(architecure_dir, 'models')
if not os.path.exists(target_dir):
  os.mkdir(target_dir)
  
model.save(os.path.join(target_dir, 'model.h5'))
model.save_weights(os.path.join(target_dir, 'weights.h5'))

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
33/33 [==============================] - 302s 9s/step - loss: 1.1286 - acc: 0.6638 - val_loss: 11.3959 - val_acc: 0.2812
Epoch 2/100
33/33 [==============================] - 18s 540ms/step - loss: 0.6573 - acc: 0.7896 - val_loss: 1.1379 - val_acc: 0.7034
Epoch 3/100
33/33 [==============================] - 13s 403ms/step - loss: 0.4436 - acc: 0.8373 - val_loss: 1.4727 - val_acc: 0.6059
Epoch 4/100
33/33 [==============================] - 13s 402ms/step - loss: 0.4658 - acc: 0.8408 - val_loss: 1.3551 - val_acc: 0.5720
Epoch 5/100
33/33 [==============================] - 13s 398ms/step - loss: 0.4297 - acc: 0.8372 - val_loss: 0.2154 - val_acc: 0.9280
Epoch 6/100
33/33 [==============================] - 13s 406ms/step - loss: 0.4146 - acc: 0.8491 - val_loss: 0.6999 - val_acc: 0.7458
Epoch 7/100
33/33 [==============================] - 13s 404ms/step - loss: 0.3461 - acc: 0.8729 - val_loss: 0.4699 - val_acc: 0.8517
Epoch 8/100
33/